# Spark DataFrames Project Exercies

In [1]:
import findspark
findspark.init()
import pyspark
findspark.find()

'D:\\Spark\\spark-3.2.1-bin-hadoop3.2'

### Import spark Session 

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Project').getOrCreate()

In [4]:
df = spark.read.csv('./doc/Spark_DataFrame_Project_Exercise/walmart_stock.csv',
                   inferSchema = True, header = True)

#### What does the Schema look like?

In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### What are the column names?

In [10]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### Print out the first 5 columns

In [14]:
df.head(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use discribe() to learn about the DataFrame

In [16]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar

In [18]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [19]:
from pyspark.sql.functions import format_number

In [21]:
result = df.describe()
result.select(result['summary'],
             format_number(result['Open'].cast('float'),2).alias('Opne'),
             format_number(result['High'].cast('float'),2).alias('High'),
             format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Opne|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [25]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [29]:
df2 = df.withColumn('HV Ratio', df['High']/df['Volume'])
df2.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in price?

In [37]:
df.orderBy(df['High'], ascending=False).select('Date').head(1)

[Row(Date='2015-01-13')]

In [36]:
df.orderBy(df['High'].desc()).head(1)[0][0]

'2015-01-13'

#### What is the mean of the Close column?

In [39]:
from pyspark.sql.functions import mean

In [40]:
df.select(mean(df['Close'])).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the volume column 

In [41]:
from pyspark.sql.functions import max, min

In [44]:
df.select([max('Volume'), min('Volume')]).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars

In [50]:
df.filter('Close < 60').count()

81

#### What percentage of the time was the High greater than 80 dollars

In [51]:
(df.filter(df['High']>80).count()*1.0/df.count())*100

9.141494435612083

#### What is the Pearson correlation between High and Volume?

In [52]:
from pyspark.sql.functions import corr
df.select(corr('High', 'Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [53]:
from pyspark.sql.functions import year
yeardf = df.withColumn('Year', year(df['Date']))

In [54]:
max_df = yeardf.groupBy('Year').max()

In [55]:
max_df.select('Year', 'max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month

In [56]:
from pyspark.sql.functions import month
monthdf= df.withColumn('Month', month(df['Date']))

In [61]:
average_df = monthdf.groupBy('Month').mean()
average_df.show(5)

+-----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------+
|Month|        avg(Open)|        avg(High)|         avg(Low)|       avg(Close)|      avg(Volume)|   avg(Adj Close)|avg(Month)|
+-----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------+
|   12|72.87952850943395|73.35566025471698|72.44481152830188|72.84792478301885|7967959.433962264|68.46031040566042|      12.0|
|    1|71.40811884158416|71.97009924752473|70.90425712871289|71.44801958415842|8761851.485148516|65.56887865346533|       1.0|
|    6| 72.5100938962264| 72.9262265471698|72.12198099056603| 72.4953774245283|8303756.603773585|67.43827906603772|       6.0|
|    3|71.69046716822429|72.20289709345795|71.31878489719628|71.77794377570092|7721836.448598131| 66.2763403084112|       3.0|
|    5|72.24349083962262|72.71783049056604|71.85292466981134|72.30971688679247|8632350.943396226|       67.0655

In [60]:
average_df.select('Month', 'avg(Close)').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|   12|72.84792478301885|
|    1|71.44801958415842|
|    6| 72.4953774245283|
|    3|71.77794377570092|
|    5|72.30971688679247|
|    9|72.18411785294116|
|    4|72.97361900952382|
|    8|73.02981855454546|
|    7|74.43971943925233|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|    2|  71.306804443299|
+-----+-----------------+

